In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

def conv_norm(in_channels, out_channels, kernel_size=3, act=True):
    '''Cria uma camada conv->batchnorm com uma ativação relu opcional.'''

    layer = [
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, 
                padding=kernel_size//2, bias=False),
        nn.BatchNorm2d(out_channels)
    ]
    if act:
        layer += [nn.ReLU()]
    
    return nn.Sequential(*layer)

class DecoderBlock(nn.Module):
    '''Recebe a ativação do nível anterior do decoder `x_dec` e a ativação do 
    encoder `x_enc`. É assumido que `x_dec` possui uma resolução espacial
    menor que `x_enc` e que `x_enc` possui número de canais diferente
    de `x_dec`.
    
    O módulo ajusta a resolução de `x_dec` para ser igual a `x_enc` e o número
    de canais de `x_enc` para ser igual a `x_dec`.'''

    def __init__(self, enc_channels, dec_channels):
        super().__init__()
        self.channel_adjust = conv_norm(enc_channels, dec_channels, kernel_size=1,
                                        act=False)
        self.mix = conv_norm(dec_channels, dec_channels)

    def forward(self, x_enc, x_dec):
        x_dec_int = F.interpolate(x_dec, size=x_enc.shape[-2:], mode="nearest")
        x_enc_ad = self.channel_adjust(x_enc)
        y = x_dec_int + x_enc_ad
        return self.mix(y)

class Decoder(nn.Module):
    '''Na criação da instância, recebe uma lista com o número de canais das
    ativações do codificador. Essa lista é necessária para criação das
    camadas de convolução. O método .forward irá receber uma lista de tensores
    e gerar uma saída com a resolução do primeiro tensor e número de canais
    dado por `decoder_channels`. 
    
    Por exemplo, suponha que as ativações extraídas de um codificador possuem 
    as dimensões:
    
    [(64,112,112), (128,56,56), (256,28,28), (512,14,14)]

    Então devemos usar `encoder_channels_list=[64, 128, 256, 512]`, e o método
    .forward irá gerar um tensor de tamanho (`decoder_channels`,112,112).
    '''

    def __init__(self, encoder_channels_list, decoder_channels):
        super().__init__()

        # Inverte lista para facilitar interpretação
        encoder_channels_list = encoder_channels_list[::-1]

        self.middle = conv_norm(encoder_channels_list[0], decoder_channels)
        blocks = []
        for channels in encoder_channels_list[1:]:
            blocks.append(DecoderBlock(channels, decoder_channels))
        self.blocks = nn.ModuleList(blocks)

    def forward(self, features):

        # Inverte lista para facilitar interpretação
        features = features[::-1]

        x = self.middle(features[0])
        for idx in range(1, len(features)):
            # Temos um bloco a menos do que nro de features, por isso
            # o idx-1
            x = self.blocks[idx-1](features[idx], x)

        return x

encoder_channels_list = [64, 128, 256]
decoder_channels = 64

decoder = Decoder(encoder_channels_list, decoder_channels)
# Lista de atributos de teste, representando os atributos extraídos de um
# codificador
x = [
    torch.rand(1, 64, 112, 112), 
    torch.rand(1, 128, 56, 56), 
    torch.rand(1, 256, 28, 28)
]
res = decoder(x)
res.shape

In [ ]:
from torchvision import models

class EncoderDecoder(nn.Module):
    """Amostra ativações de um modelo ResNet do Pytorch e cria um decodificador."""

    def __init__(self, resnet_encoder, decoder_channels, num_classes):
        super().__init__()

        # Codificador
        self.resnet_encoder = resnet_encoder
        # Extrai lista de canais dos atributos do codificador para criação de
        # decodificador
        encoder_channels_list = [64,128,256,512]
        # Decodificador
        self.decoder = Decoder(encoder_channels_list, decoder_channels)
        self.classification == nn.Convd2d(decoder_channels, num_classes, 3, padding=1)
        
        
    def get_features(self, x):
        model = self.resnet_model
        features = []
        x = model.conv1(x)
        x = model.bn1(x)
        x = model.relu(x)
        features.append(x)
        
        

In [ ]:
import torchvision.models as models